In [1]:

# Import common dependencies
import pandas as pd  # noqa
import numpy as np
import matplotlib  # noqa
import matplotlib.pyplot as plt
import datetime  # noqa
import PIL  # noqa
import glob  # noqa
import pickle  # noqa
from pathlib import Path  # noqa
from scipy import misc  # noqa
import sys
import tensorflow as tf
import pdb
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
TRADE_COST_FRAC = .003
EPSILON = 1e-10
ADV_MULT = 1e-3

In [2]:
uni_tokens = set()
uni_commands = set()
uni_actions = set()
fname = 'tasks_with_length_tags.txt'
with open(fname) as f:
    content = f.readlines()
content2 = [c.split(' ') for c in content]
# you may also want to remove whitespace characters like `\n` at the end of each line
commands = []
actions = []
content = [l.replace('\n', '') for l in content]
commands = [x.split(':::')[1].split(' ')[1:-1] for x in content]
actions = [x.split(':::')[2].split(' ')[1:-2] for x in content]
structures = [x.split(':::')[3].split(' ')[2:] for x in content]

structures = [[int(l) for l in program] for program in structures]
#actions = [[wd.replace('\n', '') for wd in res] for res in actions]

In [3]:
max_actions_per_subprogram = max([max([s for s in struct]) for struct in structures]) + 1
max_num_subprograms = max([len(s) for s in structures]) + 1
max_cmd_len = max([len(s) for s in commands]) + 1
max_act_len = max([len(a) for a in actions]) + 1
cmd_lengths_list = [len(s)+1 for s in commands]
cmd_lengths_np = np.array(cmd_lengths_list)
max_num_subprograms, max_cmd_len, max_act_len, max_actions_per_subprogram


(7, 10, 49, 9)

In [4]:
def build_fmap_invmap(unique, num_unique):
    fmap = dict(zip(unique, range(num_unique)))
    invmap = dict(zip(range(num_unique), unique))
    return fmap, invmap


In [5]:
for li in content2:
    for wd in li:
        uni_tokens.add(wd)
for li in commands:
    for wd in li:
        uni_commands.add(wd)
for li in actions:
    for wd in li:
        uni_actions.add(wd)
uni_commands.add('end_command')
uni_actions.add('end_subprogram')
uni_actions.add('end_action')
num_cmd = len(uni_commands)
num_act = len(uni_actions)
command_map, command_invmap = build_fmap_invmap(uni_commands, num_cmd)
action_map, action_invmap = build_fmap_invmap(uni_actions, num_act)

In [6]:


def dense_scaled(prev_layer, layer_size, name=None, reuse=False, scale=1.0):
    output = tf.layers.dense(prev_layer, layer_size, reuse=reuse) * scale
    return output


def dense_relu(dense_input, layer_size, scale=1.0):
    dense = dense_scaled(dense_input, layer_size, scale=scale)
    output = tf.nn.leaky_relu(dense)

    return output

def get_grad_norm(opt_fcn, loss):
    gvs = opt_fcn.compute_gradients(loss)
    grad_norm = tf.sqrt(tf.reduce_sum(
        [tf.reduce_sum(tf.square(grad)) for grad, var in gvs if grad is not None]))
    return grad_norm


def apply_clipped_optimizer(opt_fcn, loss, clip_norm=.1, clip_single=.03, clip_global_norm=False):
    gvs = opt_fcn.compute_gradients(loss)

    if clip_global_norm:
        gs, vs = zip(*[(g, v) for g, v in gvs if g is not None])
        capped_gs, grad_norm_total = tf.clip_by_global_norm([g for g in gs], clip_norm)
        capped_gvs = list(zip(capped_gs, vs))
    else:
        grad_norm_total = tf.sqrt(
            tf.reduce_sum([tf.reduce_sum(tf.square(grad)) for grad, var in gvs if grad is not None]))
        capped_gvs = [(tf.clip_by_value(grad, -1 * clip_single, clip_single), var)
                      for grad, var in gvs if grad is not None]
        capped_gvs = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in capped_gvs if grad is not None]

    optimizer = opt_fcn.apply_gradients(capped_gvs)

    return optimizer, grad_norm_total


def mlp(x, hidden_sizes, output_size=None, name='', reuse=False):
    prev_layer = x

    for idx, l in enumerate(hidden_sizes):
        dense = dense_scaled(prev_layer, l, name='mlp' + name + '_' + str(idx))
        prev_layer = tf.nn.leaky_relu(dense)

    output = prev_layer

    if output_size is not None:
        output = dense_scaled(prev_layer, output_size, name='mlp' + name + 'final')

    return output

def mlp_with_adversaries(x, hidden_sizes, output_size=None, name='', reuse=False):
    prev_layer = x
    adv_phs = []
    for idx, l in enumerate(hidden_sizes):
        
        adversary = tf.placeholder_with_default(tf.zeros_like(prev_layer), prev_layer.shape)
        prev_layer = prev_layer + adversary
        adv_phs.append(adversary)
        
        dense = dense_scaled(prev_layer, l, name='mlp' + name + '_' + str(idx))
        prev_layer = tf.nn.leaky_relu(dense)

    output = prev_layer

    if output_size is not None:
        output = dense_scaled(prev_layer, output_size, name='mlp' + name + 'final')

    return output, adv_phs



In [7]:

commands_ind = [[command_map[c] for c in cmd] + [0] * (max_cmd_len - len(cmd)) for cmd in commands]
actions_ind = [[action_map[a] for a in act] + [0] * (max_act_len - len(act)) for act in actions]
cmd_np = np.array(commands_ind)
actions_structured = []
mask_structured = []
for row in range(len(structures)):
    mask_row = []
    action_row = []
    act = actions_ind[row]
    struct = structures[row]
    start = 0
    for step in struct:
        end = start + step
        a = act[start:end]
        padding = max_actions_per_subprogram - step - 1
        action_row.append(a + [action_map['end_action']] + [0] * padding)
        start = end
    actions_structured.append(
        action_row + [[action_map['end_subprogram']] + [0] * (max_actions_per_subprogram - 1)] +
        [[0] * max_actions_per_subprogram] * (max_num_subprograms - len(struct) - 1)
    )
act_np = np.array(actions_structured)
struct_padded = [[sa + 1 for sa in s] + [1] + [0] * (max_num_subprograms - len(s) - 1) for s in structures]
struct_np = np.array(struct_padded)

mask_list = [[np.concatenate((np.ones(st), np.zeros(max_actions_per_subprogram - st)), 0) 
              for st in s] for s in struct_np]
mask_np = np.array(mask_list)

In [8]:
tf.reset_default_graph()
default_sizes = 128
size_emb = 64
num_layers_encoder = 6
hidden_filters = 128
num_layers_subprogram = 3
hidden_filters_subprogram = 128
init_mag = 1e-3
cmd_mat = tf.Variable(init_mag*tf.random_normal([num_cmd, size_emb]))
act_mat = tf.Variable(init_mag*tf.random_normal([num_act, size_emb]))
act_st_emb = tf.Variable(init_mag*tf.random_normal([size_emb]))
global_bs = None
global_time_len = 7
action_lengths = None
max_num_actions= None
# global_bs = 8
global_time_len = 7
max_num_actions = 9
output_keep_prob = tf.placeholder_with_default(1.0, ())
state_keep_prob = tf.placeholder_with_default(1.0, ())
cmd_ind = tf.placeholder(tf.int32, shape=(global_bs, 10,))
act_ind = tf.placeholder(tf.int32, shape=(global_bs, global_time_len, 9))
mask_ph = tf.placeholder(tf.float32, shape=(global_bs, global_time_len, 9))
cmd_lengths = tf.placeholder(tf.int32, shape=(global_bs,))
act_lengths = tf.placeholder(tf.int32, shape=(global_bs, 7))
learning_rate = tf.placeholder(tf.float32, shape = (None))

cmd_emb = tf.nn.embedding_lookup(cmd_mat, cmd_ind)
act_emb = tf.nn.embedding_lookup(act_mat, act_ind)
tf_bs = tf.shape(act_ind)[0]
act_st_emb_expanded = tf.tile(tf.reshape(
    act_st_emb, [1, 1, 1, size_emb]), [tf_bs, global_time_len, 1, 1])
act_emb_with_st = tf.concat((act_st_emb_expanded, act_emb), 2)

first_cell_encoder = [tf.nn.rnn_cell.LSTMCell(
    hidden_filters, forget_bias=1., name = 'layer1_'+d) for d in ['f', 'b']]
hidden_cells_encoder = [[tf.nn.rnn_cell.LSTMCell(
    hidden_filters,forget_bias=1., name = 'layer' + str(lidx) + '_' + d)  for d in ['f', 'b']]
                        for lidx in range(num_layers_encoder - 1)]
hidden_cells_encoder = [[tf.nn.rnn_cell.DropoutWrapper(cell,
    output_keep_prob=output_keep_prob, state_keep_prob=state_keep_prob,
    variational_recurrent=True, dtype=tf.float32) for cell in cells] 
                        for cells in hidden_cells_encoder[:-1]] + [hidden_cells_encoder[-1]]
cells_encoder = [first_cell_encoder] + hidden_cells_encoder
c1, c2 = zip(*cells_encoder)
cells_encoder = [c1, c2]
def encode(x, num_layers, cells, initial_states, lengths, name='',):
    prev_layer = x
    shortcut = x
    hiddenlayers = []
    returncells = []
    cell_fw, cell_bw = cells
    bs = tf.shape(x)[0]
    for idx in range(num_layers):
        prev_layer, c = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = cell_fw[idx],
                cell_bw = cell_bw[idx],
                inputs = prev_layer,
                sequence_length=lengths,
                initial_state_fw=None,
                initial_state_bw=None,
                dtype=tf.float32,
                scope='encoder'+str(idx)
            )
        prev_layer = tf.concat(prev_layer, 2)
        prev_layer = tf.nn.leaky_relu(prev_layer)
        returncells.append(c)
        hiddenlayers.append(prev_layer)
        if idx == num_layers - 1:
            #pdb.set_trace()
            output = tf.gather_nd(
                        prev_layer,
                        tf.stack([tf.range(bs), lengths], 1),
                        name=None
                    )
            return prev_layer, returncells, hiddenlayers, output
        prev_layer = tf.concat((prev_layer, shortcut), 2)
encoding_last_layer, encoding_final_cells, encoding_hidden_layers, encoding_last_timestep = encode(
    cmd_emb, num_layers_encoder, cells_encoder,None, lengths = cmd_lengths, name = 'encoder')
# encoding_last_timestep = encoding_last_layer[:,cmd_lengths, :]
hidden_filters_encoder = encoding_last_timestep.shape[-1].value
first_cell_subprogram = tf.nn.rnn_cell.LSTMCell(
    hidden_filters_subprogram, forget_bias=1., name = 'subpogramlayer1_')
hidden_cells_subprogram = [tf.nn.rnn_cell.LSTMCell(
    hidden_filters_subprogram,forget_bias=1., name = 'subpogramlayer' + str(lidx))
                        for lidx in range(num_layers_subprogram - 1)]

cells_subprogram_rnn = [first_cell_subprogram] + hidden_cells_subprogram

attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
    num_units=hidden_filters_encoder, memory=encoding_last_layer,
    memory_sequence_length=cmd_lengths)
attention_mechanism = tf.contrib.seq2seq.LuongAttention(
    num_units=hidden_filters_encoder//2, memory=encoding_last_layer,
    memory_sequence_length=cmd_lengths)
cells_subprogram = [
    tf.contrib.seq2seq.AttentionWrapper(
        cell, attention_mechanism, attention_layer_size = hidden_filters_subprogram) 
    for cell in cells_subprogram_rnn]

def subprogram(x, num_layers, cells, initial_states, lengths, reuse, name='',):
    prev_layer = x
    shortcut = x
    hiddenlayers = []
    returncells = []
    bs = tf.shape(x)[0]
    for idx in range(num_layers):
        print(idx)
        if idx == 0:
            num_past_units = hidden_filters
        else:
            num_past_units = hidden_filters_subprogram
        with tf.variable_scope(name + 'subprogram' + str(idx), reuse=reuse):
#             self_attention_mechanism = tf.contrib.seq2seq.LuongAttention(
#                 num_units=num_past_units, memory=prev_layer,
#                 memory_sequence_length=tf.expand_dims(tf.range(10), 0))
#             cell_with_selfattention = tf.contrib.seq2seq.AttentionWrapper(
#                     cells[idx], self_attention_mechanism, attention_layer_size = num_past_units)

            prev_layer, c = tf.nn.dynamic_rnn(
                    cell = cells[idx],
                    inputs = prev_layer,
                    sequence_length=lengths,
                    initial_state = None,
                    dtype=tf.float32,
                )
            prev_layer = tf.concat(prev_layer, 2)
            prev_layer = tf.nn.leaky_relu(prev_layer)
            returncells.append(c)
            hiddenlayers.append(prev_layer)
            if idx == num_layers - 1:
                output = tf.gather_nd(
                            prev_layer,
                            tf.stack([tf.range(bs), lengths], 1),
                            name=None
                        )
                return prev_layer, returncells, hiddenlayers, output
            prev_layer = tf.concat((prev_layer, shortcut), 2)
encodings = [encoding_last_timestep]
last_encoding = encoding_last_timestep
initial_cmb_encoding = last_encoding
loss = 0
action_probabilities_presoftmax = []
for sub_idx in range(max_num_subprograms): 
    from_last_layer = tf.tile(tf.expand_dims(tf.concat((
        initial_cmb_encoding, last_encoding), 1), 1), [1, max_num_actions + 1, 1])
    autoregressive = act_emb_with_st[:,sub_idx, :, :]
    x_input = tf.concat((from_last_layer, autoregressive), -1)
    subprogram_last_layer, _, subprogram_hidden_layers, subprogram_output = subprogram(
        x_input, num_layers_subprogram, cells_subprogram,None, 
        lengths = act_lengths[:, sub_idx], reuse = (sub_idx > 0), name = 'subprogram')
    action_prob_flat = mlp(
        tf.reshape(subprogram_last_layer, [-1, hidden_filters_subprogram]),
        [], output_size = num_act, name = 'action_choice_mlp', reuse = (sub_idx > 0))
    action_prob_expanded = tf.reshape(action_prob_flat, [-1, max_num_actions + 1, num_act])
    action_probabilities_layer = tf.nn.softmax(action_prob_expanded, axis=-1)
    action_probabilities_presoftmax.append(action_prob_expanded)
    delta1, delta2 = [mlp(
        subprogram_output, [256,], output_size = hidden_filters_encoder, name = 'global_transform' + str(idx),
        reuse = (sub_idx > 0)
    ) for idx in range(2)]
    remember = tf.sigmoid(delta1)
    insert = tf.tanh(delta2) + delta2/100
    last_encoding = last_encoding * remember + insert
    encodings.append(last_encoding)
    encodings.append(last_encoding)
act_presoftmax = tf.stack(action_probabilities_presoftmax, 1)[:, :, :-1, :]
#batch, subprogram, timestep, action_selection
logprobabilities = tf.nn.log_softmax(act_presoftmax, -1)
act_presoftmax_flat = tf.reshape(act_presoftmax, [-1, 9, num_act])
mask_ph_flat = tf.reshape(mask_ph, [-1, max_actions_per_subprogram])
act_ind_flat = tf.reshape(act_ind, [-1, max_actions_per_subprogram])
ppl_loss = tf.contrib.seq2seq.sequence_loss(
    logits = act_presoftmax_flat,
    targets = act_ind_flat,
    weights = mask_ph_flat,
    average_across_timesteps=False,
    average_across_batch=False,
    softmax_loss_function=None,
    name=None
)
ppl_loss_avg = tf.reduce_mean(tf.pow(ppl_loss, 2.0)) * 10000 # + tf.reduce_mean(tf.pow(ppl_loss, 1.0)) * 100

tfvars = tf.trainable_variables()
weight_norm = tf.reduce_mean([tf.reduce_sum(tf.square(var)) for var in tfvars])*1e-3

action_taken = tf.argmax(act_presoftmax, -1, output_type=tf.int32)
correct_mat = tf.cast(tf.equal(action_taken, act_ind), tf.float32) * mask_ph
correct_percent = tf.reduce_sum(correct_mat, [1, 2])/tf.reduce_sum(mask_ph, [1, 2])
percent_correct = tf.reduce_mean(correct_percent)
percent_fully_correct = tf.reduce_mean(tf.cast(tf.equal(correct_percent, 1), tf.float32))

loss = ppl_loss_avg + weight_norm

opt_fcn = tf.train.AdamOptimizer(learning_rate=learning_rate)
#opt_fcn = tf.train.MomentumOptimizer(learning_rate=learning_rate, use_nesterov=True, momentum=.8)
optimizer, grad_norm_total = apply_clipped_optimizer(opt_fcn, loss)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [9]:
hidden_filters

128

In [10]:
encoding_last_layer

<tf.Tensor 'LeakyRelu_5/Maximum:0' shape=(?, 10, 256) dtype=float32>

In [ ]:
np.random.seed(0)
trn_percent = .1
num_samples = mask_np.shape[0]
ordered_samples = np.arange(num_samples)
np.random.shuffle(ordered_samples)
trn_samples = ordered_samples[:int(np.ceil(num_samples*trn_percent))]
val_samples_all = ordered_samples[int(np.ceil(num_samples*trn_percent)):]
val_samples = val_samples_all
trn_samples.shape, val_samples.shape

((2091,), (18819,))

In [ ]:
eval_itr = -1
bs = 32# trn_samples.shape[0]
for itr in range(1000000):
    if 1:#itr == 0:
        samples = np.random.choice(trn_samples, size = bs, replace = False)
        trn_feed_dict = {
            cmd_ind : cmd_np[samples],
            act_ind : act_np[samples],
            mask_ph : mask_np[samples],
            act_lengths : np.clip(struct_np[samples], a_min = 1, a_max = None),
            cmd_lengths : cmd_lengths_np[samples],
        }
        
    trn_feed_dict[learning_rate] = .02 / (np.power(itr + 10, .6))
    _, trn_loss, acc_trn_single, acc_trn = sess.run(
        [optimizer, loss, percent_correct, percent_fully_correct], trn_feed_dict)
    if itr == 0:
        trn_loss_avg = trn_loss
        acc_trn_avg = acc_trn
        acc_trn_single_avg = acc_trn_single
    else:
        trn_loss_avg = trn_loss_avg * .9 + trn_loss * .1
        acc_trn_avg = acc_trn_avg * .9 + acc_trn * .1
        acc_trn_single_avg = acc_trn_single_avg * .9 + acc_trn_single * .1
    if itr % 10 == 0 and itr > 0:
        # val_samples = np.random.choice(val_samples_all, size = bs, replace = False)
        eval_itr += 1
        val_feed_dict = {
            cmd_ind : cmd_np[val_samples],
            act_ind : act_np[val_samples],
            mask_ph : mask_np[val_samples],
            act_lengths : np.clip(struct_np[val_samples], a_min = 1, a_max = None),
            cmd_lengths : cmd_lengths_np[val_samples]
        }
        val_loss, acc_val = sess.run([loss, percent_fully_correct], val_feed_dict)
        if eval_itr == 0:
            val_loss_avg = val_loss
            acc_val_avg = acc_val
        else:
            val_loss_avg = val_loss_avg * .9 + val_loss * .1
            acc_val_avg = acc_val_avg * .9 + acc_val * .1
        print('itr:', itr, 'trn_loss', trn_loss_avg, 'val_loss', val_loss_avg)
        print('itr:', itr, 'trn_acc', acc_trn_avg, 
              'trn_single_acc', acc_trn_single_avg, 'val_acc', acc_val_avg)

itr: 10 trn_loss 12670.714608526556 val_loss 10794.674
itr: 10 trn_acc 0.0 trn_single_acc 0.21266845135996582 val_acc 0.0
itr: 20 trn_loss 10929.766791831666 val_loss 10694.913671875
itr: 20 trn_acc 0.0 trn_single_acc 0.3098959500100507 val_acc 0.0
itr: 30 trn_loss 9787.754303839538 val_loss 10494.654433593749
itr: 30 trn_acc 0.0 trn_single_acc 0.35986784194422206 val_acc 0.0
itr: 40 trn_loss 8597.267765286475 val_loss 10207.459107421873
itr: 40 trn_acc 0.0 trn_single_acc 0.43956615311893604 val_acc 0.0
itr: 50 trn_loss 7680.608670658811 val_loss 10048.487806054685
itr: 50 trn_acc 0.0 trn_single_acc 0.4874460024648975 val_acc 0.0
itr: 60 trn_loss 7055.987554147553 val_loss 9668.224035214842
itr: 60 trn_acc 0.0 trn_single_acc 0.4939362904191626 val_acc 6.376534583978355e-05
itr: 70 trn_loss 6305.486737890167 val_loss 9252.047285990233
itr: 70 trn_acc 0.0 trn_single_acc 0.5398829655254158 val_acc 5.7388811255805195e-05
itr: 80 trn_loss 5482.713362543406 val_loss 8810.919852313085
itr: 80

itr: 530 trn_loss 1224.261674144236 val_loss 1579.7911043881093
itr: 530 trn_acc 0.036332175139914866 trn_single_acc 0.7751416003626241 val_acc 0.015024205825778104
itr: 540 trn_loss 1186.9828568362436 val_loss 1556.7359685586732
itr: 540 trn_acc 0.034228912778225526 trn_single_acc 0.7878145815583353 val_acc 0.01564198299382829
itr: 550 trn_loss 1163.891445900282 val_loss 1525.394207640306
itr: 550 trn_acc 0.03859055047008065 trn_single_acc 0.794567462788981 val_acc 0.016490240223419994
itr: 560 trn_loss 1177.3711344632304 val_loss 1493.830018809869
itr: 560 trn_acc 0.03622518078425805 trn_single_acc 0.7848314042417618 val_acc 0.01744496780831724
itr: 570 trn_loss 1171.8173041245127 val_loss 1471.1170730812257
itr: 570 trn_acc 0.03656665674069561 trn_single_acc 0.777122435495798 val_acc 0.01853271499156156
itr: 580 trn_loss 1200.2265905143188 val_loss 1453.042121144197
itr: 580 trn_acc 0.02836381545389291 trn_single_acc 0.7852619534546517 val_acc 0.019129095486216538
itr: 590 trn_loss 

itr: 1040 trn_loss 797.2014388140714 val_loss 1004.9064264812582
itr: 1040 trn_acc 0.06307037270932879 trn_single_acc 0.8393739228543639 val_acc 0.04457844041984761
itr: 1050 trn_loss 791.189985507055 val_loss 998.0335206495387
itr: 1050 trn_acc 0.0591733212884394 trn_single_acc 0.8415471808743279 val_acc 0.04532278583825962
itr: 1060 trn_loss 796.5740804151649 val_loss 991.5295887506004
itr: 1060 trn_acc 0.06269192607801419 trn_single_acc 0.8361536497766553 val_acc 0.046247758028340365
itr: 1070 trn_loss 781.3578468214426 val_loss 989.4692018970247
itr: 1070 trn_acc 0.06640539888549651 trn_single_acc 0.8338250822523309 val_acc 0.046368186599700476
itr: 1080 trn_loss 788.1170821379299 val_loss 987.7344399104472
itr: 1080 trn_acc 0.07865050307886326 trn_single_acc 0.8376332882690931 val_acc 0.046524396100660115
itr: 1090 trn_loss 776.1467159970833 val_loss 996.2662815639338
itr: 1090 trn_acc 0.0769384895391183 trn_single_acc 0.8450005647796824 val_acc 0.04650557128401394
itr: 1100 trn_l

itr: 1540 trn_loss 589.8650255488138 val_loss 838.6778095493414
itr: 1540 trn_acc 0.11671947222462659 trn_single_acc 0.8709730972734925 val_acc 0.06240915890509357
itr: 1550 trn_loss 595.141477738939 val_loss 834.818042510423
itr: 1550 trn_acc 0.0956503447733281 trn_single_acc 0.8687764499172591 val_acc 0.062241891869674354
itr: 1560 trn_loss 555.4676403080532 val_loss 836.1087480250058
itr: 1560 trn_acc 0.08979575018871624 trn_single_acc 0.8766827147872106 val_acc 0.06195319355914315
itr: 1570 trn_loss 588.3810593243542 val_loss 833.7727023240677
itr: 1570 trn_acc 0.1034198076815359 trn_single_acc 0.8701632429390876 val_acc 0.0621025256340741
itr: 1580 trn_loss 585.3442343257171 val_loss 828.7121496209578
itr: 1580 trn_acc 0.10300315807408997 trn_single_acc 0.8687828174182475 val_acc 0.06285863670930614
itr: 1590 trn_loss 553.4467488074444 val_loss 825.5143965729245
itr: 1590 trn_acc 0.11459905116702244 trn_single_acc 0.8766112501367493 val_acc 0.06337440955542341
itr: 1600 trn_loss 5

itr: 2050 trn_loss 449.79969088570624 val_loss 773.0423596842633
itr: 2050 trn_acc 0.15427914356774886 trn_single_acc 0.8924221470624589 val_acc 0.07723207624919258
itr: 2060 trn_loss 428.02879424476856 val_loss 774.3724743017744
itr: 2060 trn_acc 0.17014433485354166 trn_single_acc 0.8966114646886427 val_acc 0.07764957713038066
itr: 2070 trn_loss 446.98571921669566 val_loss 776.4292332192533
itr: 2070 trn_acc 0.14007226729045993 trn_single_acc 0.8936111272556643 val_acc 0.07790311157957024
itr: 2080 trn_loss 454.59359566782945 val_loss 775.2257325047499
itr: 2080 trn_acc 0.14626777280385783 trn_single_acc 0.8928099983065895 val_acc 0.07845543307294225
itr: 2090 trn_loss 465.50592442370294 val_loss 773.3535010511499
itr: 2090 trn_acc 0.13311389647690036 trn_single_acc 0.8931192804492158 val_acc 0.07876654027905859
itr: 2100 trn_loss 460.3973858059753 val_loss 773.8464773620506
itr: 2100 trn_acc 0.13153742107919852 trn_single_acc 0.8960103774035686 val_acc 0.07909967430538056
itr: 2110 t

...

itr: 39680 trn_loss 0.10546468798722994 val_loss 4332.502611374332
itr: 39680 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.6981842188434012
itr: 39690 trn_loss 0.10541941994053063 val_loss 4332.455426408775
itr: 39690 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.6981410266515202
itr: 39700 trn_loss 0.10538472905681405 val_loss 4333.382540017898
itr: 39700 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.6981127811869908
itr: 39710 trn_loss 0.10536276748758408 val_loss 4332.863866094232
itr: 39710 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.6981245535672542
itr: 39720 trn_loss 0.10532547260323455 val_loss 4333.01097557856
itr: 39720 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.6981670312339816
itr: 39730 trn_loss 0.10528409017713722 val_loss 4335.161733489454
itr: 39730 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.6981096135605651
itr: 

itr: 40180 trn_loss 0.10345878407066463 val_loss 4352.343385875233
itr: 40180 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.6971829975340332
itr: 40190 trn_loss 0.10340076014945788 val_loss 4347.886537522085
itr: 40190 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.6972080449485987
itr: 40200 trn_loss 0.10336569248553379 val_loss 4343.776936504251
itr: 40200 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.6971561891040989
itr: 40210 trn_loss 0.10331468280923904 val_loss 4341.695043635076
itr: 40210 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.6971839173616577
itr: 40220 trn_loss 0.10326358099277987 val_loss 4341.070412318443
itr: 40220 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.6971876177771338
itr: 40230 trn_loss 0.10321807744670605 val_loss 4339.14686718035
itr: 40230 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.6972600239738924
itr: 

itr: 40680 trn_loss 0.10097409633994191 val_loss 4369.914625980829
itr: 40680 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.6951175366588378
itr: 40690 trn_loss 0.10090453235101259 val_loss 4369.4656926796215
itr: 40690 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.6949771614147374
itr: 40700 trn_loss 0.10085604098474642 val_loss 4371.123469114785
itr: 40700 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.6949464712685183
itr: 40710 trn_loss 0.10079750455246778 val_loss 4373.461464000182
itr: 40710 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.6948338300716134
itr: 40720 trn_loss 0.10073841622487223 val_loss 4376.465317600164
itr: 40720 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.6947643355188894
itr: 40730 trn_loss 0.10068584571746506 val_loss 4380.723180371398
itr: 40730 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.6947443004540915
itr

itr: 41180 trn_loss 12.650801162194192 val_loss 4414.253612902372
itr: 41180 trn_acc 0.9916569925145101 trn_single_acc 0.9994975107186356 val_acc 0.6747076820554824
itr: 41190 trn_loss 5.661535205501754 val_loss 4390.39607387776
itr: 41190 trn_acc 0.9904020131329668 trn_single_acc 0.9994818319476789 val_acc 0.674961022545918
itr: 41200 trn_loss 94.02772486122161 val_loss 4399.627267271234
itr: 41200 trn_acc 0.9800109913798527 trn_single_acc 0.9988028937964609 val_acc 0.6740359413713434
itr: 41210 trn_loss 36.633253790622774 val_loss 4459.661464372235
itr: 41210 trn_acc 0.9831050017801817 trn_single_acc 0.998939542533637 val_acc 0.6734106017431811
itr: 41220 trn_loss 102.90009862684884 val_loss 4468.410698794387
itr: 41220 trn_acc 0.9864351664720965 trn_single_acc 0.9990560054305291 val_acc 0.6736129826260683
itr: 41230 trn_loss 39.837980913190826 val_loss 4407.983203133698
itr: 41230 trn_acc 0.9885532062240245 trn_single_acc 0.9992633958721951 val_acc 0.6755964969950685
itr: 41240 trn_

itr: 41680 trn_loss 1.9947816313455697 val_loss 3888.453752060937
itr: 41680 trn_acc 0.9973362775379736 trn_single_acc 0.9998433128321793 val_acc 0.7039811092434959
itr: 41690 trn_loss 0.8341232627071254 val_loss 3864.6407010735934
itr: 41690 trn_acc 0.9990712174070814 trn_single_acc 0.9999453665627407 val_acc 0.7049204776720974
itr: 41700 trn_loss 0.35662180137595206 val_loss 3858.2692335052966
itr: 41700 trn_acc 0.999676153534309 trn_single_acc 0.9999809504983191 val_acc 0.7053939405116534
itr: 41710 trn_loss 0.19005276538541133 val_loss 3851.1659185532044
itr: 41710 trn_acc 0.9998870817195109 trn_single_acc 0.9999933578494691 val_acc 0.705883828076699
itr: 41720 trn_loss 0.1319940837862056 val_loss 3853.4180278697595
itr: 41720 trn_acc 0.9999606278301002 trn_single_acc 0.9999976840253139 val_acc 0.7057136421856094
itr: 41730 trn_loss 0.12109921425358888 val_loss 3869.073002426534
itr: 41730 trn_acc 0.999986271773216 trn_single_acc 0.999999192469559 val_acc 0.7056720607391127
itr: 41

In [ ]:
correct_percent.shape, percent_fully_correct.shape

In [ ]:
10, None, 7, 9, None

In [ ]:
cmd_np.shape, act_np.shape, mask_np.shape, struct_np.shape, cmd_lengths_np.shape

In [ ]:
cmd_ind.shape, act_ind.shape, mask_ph.shape, act_lengths.shape, cmd_lengths.shape

In [ ]:
trn_samples.shape, val_samples.shape

In [ ]:

act_presoftmax = tf.stack(action_probabilities_presoftmax, 1)[:, :, :-1, :]
#batch, subprogram, timestep, action_selection
logprobabilities = tf.nn.log_softmax(act_presoftmax, -1)
act_presoftmax_flat = tf.reshape(act_presoftmax, [-1, 9, num_act])
mask_ph_flat = tf.reshape(mask_ph, [-1, max_actions_per_subprogram])
act_ind_flat = tf.reshape(act_ind, [-1, max_actions_per_subprogram])

In [ ]:
act_presoftmax_flat = tf.reshape(act_presoftmax, [-1, 9, num_act])

In [ ]:
max_actions_per_subprogram

In [ ]:
sess.run(act_presoftmax, feed_dict).shape

In [ ]:
action_map

In [ ]:
print(*actions_ind[2])

In [ ]:
command_map

In [ ]:
action_map

In [ ]:
subprogram_output

In [ ]:
subprogram_last_layer[:,cmd_lengths,:]

In [ ]:
encoding_last_layer

In [ ]:
tf.gather(
    encoding_last_layer,
    [1,2],
    axis=1
)

In [ ]:
tf.gather_nd(
    encoding_last_layer,
    np.array([[0,1,2,3,4], [1,4,3,2,5]]).T,
    name=None
)

In [ ]:
cmd_lengths

In [ ]:
def generate_command(sub_cmd, num_repeat):
    return sub_cmd * num_repeat

In [ ]:
def process_command(cmd):
    

In [ ]:
uni_commands

In [ ]:
uni_actions

In [ ]:
uni_tokens

In [ ]:
df.shape